In [1]:
#Path to Images
impath = "data/images_training_rev1"
#Path to Labels CSV
lapath = "training_solutions_rev1.csv"

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf
import pathlib
import os
import sys
%matplotlib inline

In [3]:
label_df = pd.read_csv(lapath)
label_df["GalaxyID"] = label_df["GalaxyID"].apply((lambda x: str(x) + ".jpg"))
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)

In [21]:
train_generator=datagen.flow_from_dataframe(
dataframe=label_df,
directory=impath,
x_col="GalaxyID",
y_col=list(label_df.keys())[1:],
class_mode = "raw",
subset="training",
batch_size=32,
color_mode="grayscale",
target_size=(424,424))

validation_generator=datagen.flow_from_dataframe(
dataframe=label_df,
directory=impath,
x_col="GalaxyID",
y_col=list(label_df.keys())[1:],
class_mode = "raw",
subset="validation",
batch_size=32,
color_mode="grayscale",
target_size=(424,424))

Found 55421 validated image filenames.
Found 6157 validated image filenames.


In [22]:
thing = train_generator[0]

print(thing[0].shape)

(32, 424, 424, 1)


In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(424,424,1)))
model.add(tf.keras.layers.Reshape((424*424,)))
model.add(tf.keras.layers.Rescaling(1./255, offset=0.0))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dense((37)))
model.compile(
    loss = 'mean_squared_error'
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 179776)            0         
                                                                 
 rescaling_3 (Rescaling)     (None, 179776)            0         
                                                                 
 dense_9 (Dense)             (None, 32)                5752864   
                                                                 
 dense_10 (Dense)            (None, 32)                1056      
                                                                 
 dense_11 (Dense)            (None, 37)                1221      
                                                                 
Total params: 5,755,141
Trainable params: 5,755,141
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x=train_generator)

2022-05-20 21:12:13.524485: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1732/1732 [==============================] - 124s 71ms/step - loss: 3.3571


In [25]:
inp = np.reshape(train_generator[0][0][0],(1,424,424,1))

In [26]:
model(inp)

<tf.Tensor: shape=(1, 37), dtype=float32, numpy=
array([[ 0.5023605 ,  0.43266264,  0.0225061 ,  0.1966215 ,  0.2338585 ,
         0.07673927,  0.17633027,  0.10144052,  0.16279   ,  0.01561499,
         0.10443321,  0.15198109,  0.05398866,  0.16013697,  0.7653103 ,
         0.25911143,  0.22837207,  0.09839367,  0.02298955,  0.04075496,
         0.0266229 , -0.00189123,  0.06156732, -0.02775297, -0.01161042,
         0.16036029,  0.02513487,  0.0603941 ,  0.02512203,  0.03291135,
        -0.01773255,  0.02408445,  0.0200637 ,  0.06950925, -0.01144877,
        -0.03734456,  0.06011204]], dtype=float32)>

In [31]:
np.reshape(train_generator[0][1][0],(1,37))

array([[0.265664  , 0.727164  , 0.007172  , 0.        , 0.727164  ,
        0.        , 0.727164  , 0.        , 0.727164  , 0.        ,
        0.        , 0.60496845, 0.12219555, 0.031901  , 0.968099  ,
        0.069914  , 0.17969513, 0.01605487, 0.        , 0.        ,
        0.        , 0.031901  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])